# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 目標均值編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [1]:
# 程式區塊 A
# 將需要的都import進來
import os
import copy
import time
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
import math
import warnings
from scipy                   import stats
from sklearn.preprocessing   import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model    import LogisticRegression,LinearRegression
from sklearn.model_selection import cross_val_score
# 將較長的函式改名一下
MME  = MinMaxScaler()
LR   = LogisticRegression()
LIR  = LinearRegression()
LE   = LabelEncoder()
PDDF = pd.DataFrame()
# 一些必要的設定
warnings.filterwarnings('ignore')
%matplotlib inline

# 設定【data的資料夾路徑】，命名為【data_folder】
data_folder = 'C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data'

In [2]:
# 設定t001為某個data路徑
# 設定t002為pd裡read data的功能
t001_train = os.path.join(data_folder, 'titanic_train.csv')
t002_train = pd.read_csv(t001_train)
print('Path of read in data: %s' %t001_train)
print(t002_train.shape)
t002_train.head()

Path of read in data: C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data\titanic_train.csv
(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 設定t001為某個data路徑
# 設定t002為pd裡read data的功能
t001_test  = os.path.join(data_folder,  'titanic_test.csv')
t002_test  = pd.read_csv(t001_test)
print('Path of read in data: %s' %t001_test)
print(t002_test.shape)
t002_test.head()

Path of read in data: C:/Users/Ynitsed/Documents/GitHub/2nd-ML100Days/data\titanic_test.csv
(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
# 程式區塊 B-1：train取【Survived】為Y、test取【PassengerId】為最終將比對的唯一識別
train_Y     = t002_train['Survived']
test_unique = t002_test['PassengerId']
# 程式區塊 B-2：train捨棄【PassengerId,Survived】、test捨棄【PassengerId】。
t003_train = t002_train.drop(['PassengerId', 'Survived'] , axis=1)
t003_test  = t002_test.drop(['PassengerId'] , axis=1)
print(t003_train.shape)
print(t003_test.shape)
# 程式區塊 B-3：設計t005，把train和test合併，UNION ALL的概念。
t004 = pd.concat([t003_train,t003_test])
print(t004.shape)

(891, 10)
(418, 10)
(1309, 10)


In [5]:
#只取 object 類別型欄位, 存於 num_features 中
object_features = []
for a, b in zip(t004.dtypes, t004.columns):
    if a == 'object':
        object_features.append(b)
print(f'{len(object_features)} Numeric Features : {object_features}\n')

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



In [6]:
# 削減文字型欄位, 只剩數值型欄位
t005 = t004[object_features]
# 將空值補0
t006 = t005.fillna('None')
# 計算訓練資料的筆數
train_cnt = train_Y.shape[0]
print(train_cnt)
t006.head(5)

891


,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?

In [7]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
PDDF = pd.DataFrame()
for d in t006.columns:
    PDDF[d] = LE.fit_transform(t006[d])
train_X = PDDF[:train_cnt]
start = time.time()
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(LR, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')
PDDF.head()

shape : (891, 5)
score : 0.780004837244799
time : 0.05297589302062988 sec


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [8]:
# 加上 'Cabin' 欄位的計數編碼
t007 = t006.groupby(['Cabin'])['Name'].agg({'Cabin_Count':'size'}).reset_index()
t008 = pd.merge(t006, t007, on=['Cabin'], how='left')
t007.sort_values(by=['Cabin_Count'], ascending=False).head(10)

,Cabin,Cabin_Count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [9]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
PDDF = pd.DataFrame()
for d in object_features:
    PDDF[d] = LE.fit_transform(t008[d])
PDDF['Cabin_Count'] = t008['Cabin_Count']
train_X = PDDF[:train_cnt]
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(LR, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')
PDDF.head()

shape : (891, 6)
score : 0.7856230275549181
time : 0.21195030212402344 sec


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Count
0,155,1,720,185,3,1014
1,286,0,816,106,0,2
2,523,0,914,185,3,1014
3,422,0,65,70,3,2
4,22,1,649,185,3,1014


In [10]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
PDDF = pd.DataFrame()
for d in object_features:
    PDDF[d] = LE.fit_transform(t008[d])
PDDF['Cabin_Hash'] = t008['Cabin'].map(lambda x:hash(x) % 5)
train_X = PDDF[:train_cnt]
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(LR, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')
PDDF.head()

shape : (891, 6)
score : 0.7777639233034268
time : 0.29993629455566406 sec


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,3
1,286,0,816,106,0,3
2,523,0,914,185,3,3
3,422,0,65,70,3,2
4,22,1,649,185,3,3


In [11]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
PDDF = pd.DataFrame()
for d in object_features:
    PDDF[d] = LE.fit_transform(t008[d])
PDDF['Cabin_Hash'] = t008['Cabin'].map(lambda x:hash(x) % 5)
PDDF['Cabin_Count'] = t008['Cabin_Count']
train_X = PDDF[:train_cnt]
print(f'shape : {train_X.shape}')
print(f'score : {cross_val_score(LR, train_X, train_Y, cv=5).mean()}')
print(f'time : {time.time() - start} sec')
PDDF.head()

shape : (891, 7)
score : 0.7755483304235391
time : 0.3899240493774414 sec


,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash,Cabin_Count
0,155,1,720,185,3,3,1014
1,286,0,816,106,0,3,2
2,523,0,914,185,3,3,1014
3,422,0,65,70,3,2,2
4,22,1,649,185,3,3,1014


In [ ]:
### Day24教材方向和目標
屬於延續教材，複習時再一併了解

### Day24忽略部分
.map(lambda x:hash(x) % 5)的用意

### Day24其他補充
